importing neccessary libraries 

In [1]:
import pandas as pd
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score


extracting zip file 

In [2]:
import zipfile
import os

zip_path = r"C:\Users\pande\Desktop\aml project\liar_dataset.zip"
extract_path = r"C:\Users\pande\Desktop\aml project\liar_dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


Loading data set and preprocessing 
reading train.tsv and assing columns to it manually as it does noit contain any header 

In [3]:
import pandas as pd

train_file = os.path.join(extract_path, "train.tsv")
df = pd.read_csv(train_file, sep='\t', header=None)
df.columns = ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 
              'state_info', 'party_affiliation', 'barely_true_counts', 
              'false_counts', 'half_true_counts', 'mostly_true_counts', 
              'pants_on_fire_counts', 'context']



label mapping - truth labels are assinged 1 fake as 0 , unmapped labels are dropped 

In [4]:
df = df[['statement', 'label']]
df.dropna(inplace=True)

# Map labels to binary: fake (0) and real (1)
label_mapping = {
    'true': 1, 'mostly-true': 1, 'half-true': 1,
    'barely-true': 0, 'false': 0, 'pants-fire': 0
}
df['label'] = df['label'].map(label_mapping)
df.dropna(inplace=True)  # in case of unmapped labels


DATA CLEANING 
text are made lower case , special characters are removed  only alphabets are retained , stop words (this ,is ,and ) are also removed for better model accuracy 

In [ ]:
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])
    return text

df['statement'] = df['statement'].apply(clean_text)
# cleaning func is applied on every statement 


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pande\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


text are converted to nos using td - idf 
max featues maens only top 5000 wwords are used

In [7]:
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(df['statement']).toarray()
y = df['label'].values


train test split i.e 80/20 

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Train 3 ML Models
(LR, NB, SGD) trained and their  performance evaluated  with precision, recall, F1.



In [9]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
print("Logistic Regression\n", classification_report(y_test, y_pred_lr))


Logistic Regression
               precision    recall  f1-score   support

           0       0.55      0.43      0.48       888
           1       0.63      0.73      0.67      1160

    accuracy                           0.60      2048
   macro avg       0.59      0.58      0.58      2048
weighted avg       0.59      0.60      0.59      2048



In [10]:
nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
print("Naive Bayes\n", classification_report(y_test, y_pred_nb))


Naive Bayes
               precision    recall  f1-score   support

           0       0.56      0.36      0.44       888
           1       0.61      0.78      0.69      1160

    accuracy                           0.60      2048
   macro avg       0.59      0.57      0.56      2048
weighted avg       0.59      0.60      0.58      2048



In [11]:
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
y_pred_sgd = sgd.predict(X_test)
print("SGD Classifier\n", classification_report(y_test, y_pred_sgd))


SGD Classifier
               precision    recall  f1-score   support

           0       0.53      0.49      0.51       888
           1       0.63      0.67      0.65      1160

    accuracy                           0.59      2048
   macro avg       0.58      0.58      0.58      2048
weighted avg       0.59      0.59      0.59      2048



printed down all final accuracy score 

In [12]:
from sklearn.metrics import accuracy_score

# Logistic Regression
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
acc_lr = accuracy_score(y_test, y_pred_lr)
print(f"Logistic Regression Accuracy: {acc_lr:.4f}")

# Naive Bayes
nb = MultinomialNB()
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
acc_nb = accuracy_score(y_test, y_pred_nb)
print(f"Naive Bayes Accuracy: {acc_nb:.4f}")

# SGD Classifier
sgd = SGDClassifier()
sgd.fit(X_train, y_train)
y_pred_sgd = sgd.predict(X_test)
acc_sgd = accuracy_score(y_test, y_pred_sgd)
print(f"SGD Classifier Accuracy: {acc_sgd:.4f}")


Logistic Regression Accuracy: 0.5996
Naive Bayes Accuracy: 0.5981
SGD Classifier Accuracy: 0.5918


 LSTM Based Deep Learning
 Deep learning ka setup: Tokenizer, Padding, Embedding, LSTM, Dropout layers imported


In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout


Same dataset load and preprocess steps

In [24]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load dataset
df = pd.read_csv(r"C:\Users\pande\Desktop\aml project\liar_dataset\train.tsv", sep='\t', header=None)
df.columns = ['id', 'label', 'statement', 'subject', 'speaker', 'job_title', 
              'state_info', 'party_affiliation', 'barely_true_counts', 
              'false_counts', 'half_true_counts', 'mostly_true_counts', 
              'pants_on_fire_counts', 'context']
df = df[['statement', 'label']]
df.dropna(inplace=True)

# Label encoding
label_map = {'true':1, 'mostly-true':1, 'half-true':1, 'barely-true':0, 'false':0, 'pants-fire':0}
df['label'] = df['label'].map(label_map)
df.dropna(inplace=True)

# Tokenize and pad
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['statement'])
X = tokenizer.texts_to_sequences(df['statement'])
X = pad_sequences(X, maxlen=50)

y = df['label'].values

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build LSTM model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=50))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Train the model
model.fit(X_train, y_train, epochs=15, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
loss, acc = model.evaluate(X_test, y_test)
print(f"LSTM Accuracy: {acc:.4f}")


C:\Users\pande\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
128/128 ━━━━━━━━━━━━━━━━━━━━ 25s 28ms/step - accuracy: 0.5647 - loss: 0.6800 - val_accuracy: 0.5952 - val_loss: 0.6606
Epoch 2/15
128/128 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.6887 - loss: 0.5946 - val_accuracy: 0.5918 - val_loss: 0.6874
Epoch 3/15
128/128 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.7696 - loss: 0.4898 - val_accuracy: 0.5825 - val_loss: 0.7498
Epoch 4/15
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.8501 - loss: 0.3695 - val_accuracy: 0.5811 - val_loss: 0.8624
Epoch 5/15
128/128 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.8851 - loss: 0.2907 - val_accuracy: 0.5684 - val_loss: 1.0931
Epoch 6/15
128/128 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - accuracy: 0.9201 - loss: 0.2091 - val_accuracy: 0.5811 - val_loss: 1.2219
Epoch 7/15
128/128 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.9353 - loss: 0.1673 - val_accuracy: 0.5752 - val_loss: 1.4028
Epoch 8/15
128/128 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 0.9560 - loss: 0.1272 - val_acc

In [16]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def encode(example):
    return tokenizer(
        example['text'],  # Column with the statement
        truncation=True,
        padding='max_length',
        max_length=128
    )



In [17]:
import os
print(os.getcwd())



c:\Users\pande\Desktop\aml project


In [ ]:
train_data = load_dataset('csv', data_files=r"C:\Users\pande\Desktop\aml project\liar_dataset\train.tsv", delimiter='\t')
test_data = load_dataset('csv', data_files=r"C:\Users\pande\Desktop\aml project\liar_dataset\test.tsv", delimiter='\t')


In [ ]:
print(train_data['train'].column_names)
print(test_data['train'].column_names)


In [25]:
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")


ImportError: 
BertForSequenceClassification requires the PyTorch library but it was not found in your environment.
However, we were able to find a TensorFlow installation. TensorFlow classes begin
with "TF", but are otherwise identically named to our PyTorch classes. This
means that the TF equivalent of the class you tried to import would be "TFBertForSequenceClassification".
If you want to use TensorFlow, please use TF classes instead!

If you really do want to use PyTorch please go to
https://pytorch.org/get-started/locally/ and follow the instructions that
match your environment.


In [56]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.19.0
